# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Henning  ->  T. Henning  |  ['T. Henning']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 51 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.04098


extracting tarball to tmp_2501.04098... done.
Retrieving document from  https://arxiv.org/e-print/2501.04518


extracting tarball to tmp_2501.04518...

 done.


...

Found 129 bibliographic references in tmp_2501.04587/main.bbl.
Issues with the citations
syntax error in line 95: '=' expected
Retrieving document from  https://arxiv.org/e-print/2501.04616


extracting tarball to tmp_2501.04616...

 done.


H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 97 bibliographic references in tmp_2501.04616/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04518-b31b1b.svg)](https://arxiv.org/abs/2501.04518) | **The Cygnus Allscale Survey of Chemistry and Dynamical Environments: CASCADE. IV. Unveiling the hidden structures in DR18**  |
|| W.-J. Kim, et al. -- incl., <mark>H. Beuther</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2025-01-09*|
|*Comments*| *25 pages, 21 figures, and 4 tables. Accepted for publication in A&A*|
|**Abstract**|            The Cygnus-X complex is a massive, nearby (1.4 kpc) star-forming region with several OB associations. As part of the Cygnus Allscale Survey of Chemistry and Dynamical Environments (CASCADE) program, we carried out 3.6 millimeter (mm) continuum and spectral line high-resolution observations ($\sim$ 3 - 4$''$) toward DR18, covering several molecular species with the Northern Extended Millimeter Array (NOEMA) and the Institut de Radioastronomie Millimétrique (IRAM) 30m telescope. In addition, multi-wavelength archival datasets were used to provide a comprehensive analysis of the region. A comparison of the 3.6mm and 6 cm continuum emission confirms that a B2 star (DR18-05) shapes the cometary HII region in the DR18 cavity, with ionized gas escaping toward the OB2 association. On the other hand, the extended 3.6mm and 6 cm continuum emission are likely to trace photoevaporating ionized gas from ultraviolet radiation from the Cyg OB2 association, not from DR18-05. The shell structure around DR18-05 indicates photodissociation regions (PDRs) formed by the expanding HII region and photo-erosion from DR18-05 and OB2 stars. We also identified 18 compact cores with N$_2$H$^+$ emission, half of which are gravitationally bound and mostly located in colder regions behind the PDRs. The SiO emission is found only in PDRs, with narrow-line widths ( 0.8 - 2.0 km s$^{-1}$) and lower abundances (X(SiO) $\sim$ 5$\times$10$^{-11}$ - 1$\times$10$^{-10}$). Comparing with the UV irradiated shock models, we suggest that the SiO emission partially encompassing the HII region arises from the molecular gas region, marginally compressed by low-velocity shocks with $\sim$ 5 km s$^{-1}$, irradiated by external UV radiation (G$_{\rm 0} \sim 10^{2} - 10^{3}$), as they traverse through a medium with $n_{\rm H} \sim 10^{4}$ to 10$^5$ cm$^{-3}$.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04523-b31b1b.svg)](https://arxiv.org/abs/2501.04523) | **NGTS-EB, an eccentric, long-period, low-mass eclipsing binary**  |
|| T. Rodel, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-01-09*|
|*Comments*| *Main body: 14 pages, 6 figures, 5 tables. Appendices: 7 pages, 5 figures, 5 tables*|
|**Abstract**|            Despite being the most common types of stars in the Galaxy, the physical properties of late M dwarfs are often poorly constrained. A trend of radius inflation compared to evolutionary models has been observed for earlier type M dwarfs in eclipsing binaries, possibly caused by magnetic activity. It is currently unclear whether this trend also extends to later type M dwarfs below theconvective boundary. This makes the discovery of lower-mass, fully convective, M dwarfs in eclipsing binaries valuable for testing evolutionary models especially in longer-period binaries where tidal interaction between the primary and secondary is negligible. With this context, we present the discovery of the NGTS-EB-7 AB system, an eclipsing binary containing a late M dwarf secondary and an evolved G-type primary star. The secondary star has a radius of $0.125 \pm 0.006 R_\odot$ , a mass of $0.096 \pm 0.004 M_\odot$ and follows a highly eccentric $(e=0.71436 \pm 0.00085)$ orbit every $193.35875 \pm 0.00034$ days. This makes NGTS-EB-7 AB the third longest-period eclipsing binary system with a secondary smaller than $200 M_J$ with the mass and radius constrained to better than $5 \%$. In addition, NGTS-EB-7 is situated near the centre of the proposed LOPS2 southern field of the upcoming PLATO mission, allowing for detection of the secondary eclipse and measurement of the companion`s temperature. With its long-period and well-constrained physical properties - NGTS-EB-7 B will make a valuable addition to the sample of M dwarfs in eclipsing binaries and help in determining accurate empirical mass/radius relations for later M dwarf stars.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04587-b31b1b.svg)](https://arxiv.org/abs/2501.04587) | **MINDS. The influence of outer dust disc structure on the volatile delivery to the inner disc**  |
|| D. Gasman, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-01-09*|
|*Comments*| *Accepted for publication in A&A. 29 pages, 12 figures and 10 tables*|
|**Abstract**|            ALMA has revealed that the millimetre dust structures of protoplanetary discs are extremely diverse. It has been proposed that the strength of H$_2$O emission in the inner disc particularly depends on the influx of icy pebbles from the outer disc, a process that would correlate with the outer dust disc radius, and that could be prevented by pressure bumps. This work aims to assess the influence of pressure bumps on the inner disc's molecular reservoirs. Using JWST's MIRI/MRS, we compared the observational emission properties of H$_2$O, HCN, C$_2$H$_2$, and CO$_2$ with the outer dust disc structure from ALMA observations, in eight discs with confirmed gaps in ALMA observations, and two discs with gaps of tens of astronomical units in width, around stars with $M_\star \geq 0.45M_{\odot}$. We used new visibility plane fits of the ALMA data to determine the outer dust disc radius and identify substructures in the discs. We find that the presence of a dust gap does not necessarily result in weak H$_2$O emission. Furthermore, the relative lack of colder H$_2$O-emission seems to go hand in hand with elevated emission from carbon-bearing species. The discs with cavities and extremely wide gaps appear to behave as a somewhat separate group, with stronger cold H$_2$O emission and weak warm H$_2$O emission. We conclude that fully blocking radial dust drift from the outer disc seems difficult to achieve. However, there does seem to be a dichotomy between discs that show a strong cold H$_2$O excess and ones that show strong emission from HCN and C$_2$H$_2$. Better constraints on the influence of the outer dust disc structure and inner disc composition require more information on substructure formation timescales and disc ages, along with the importance of trapping of volatiles like CO and CO$_2$ into more strongly bound ices like H$_2$O and chemical transformation of CO into less volatile species.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04616-b31b1b.svg)](https://arxiv.org/abs/2501.04616) | **Vertical structure and kinematics of the LMC disc from SDSS/Gaia**  |
|| Ó. Jiménez-Arranz, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-01-09*|
|*Comments*| *Submitted to A&A. The interactive LMC 3D models can be found in this https URL*|
|**Abstract**|            [Abbreviated] Context: Studies of the LMC's internal kinematics have provided a detailed view of its structure, largely by the exquisite proper motion data supplied by Gaia. However, LoS velocities are only available for a small subset of Gaia data, limiting studies of the kinematics perpendicular to the LMC disc plane. Aims: We synergise new SDSS-IV/V LoS velocity measurements with Gaia data, increasing the 5D phase-space sample by almost a factor of three. We interpret and model the vertical structure and kinematics of the LMC disc. Methods: Split our sample into different stellar types. Then examine maps of vertical velocity moments perpendicular to the LMC disc. We interpret our results within three possible scenarios: 1) time-variability in the orientation of the disc symmetry axis; 2) use of an incorrect LMC disc plane orientation; or 3) the presence of warps or twists in the LMC disc. We also present a new method to construct a continuous 3D representation of the disc from spatially-resolved measurements of its viewing angles. Results: Using young stellar populations, we identify a region in the LMC arm with highly negative v_z'; this overlaps spatially with the supershell LMC 4. Our results indicate that: 1) the LMC viewing angles may vary with time, but this cannot explain most of the structure in v_z' maps; 2) when re-deriving the LMC disc plane by minimising the RMS vertical velocity v_z' across the disc, the inclination and line-of-nodes position angle are i ~ 24 degr and \Omega ~ 327 degr, respectively; 3) we obtain different inclinations for the inner and outer disc regions, and a quadrupolar variation with azimuth in outer the disc. We provide 3D models of the LMC disc shape. Conclusions: The combination of SDSS-IV/V and Gaia data reveal that the LMC disc is not a flat plane in equilibrium, but that the central bar region is tilted relative to a warped outer disc.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04098-b31b1b.svg)](https://arxiv.org/abs/2501.04098) | **Revisiting the Classics: On the Statistics of Dust Formation in Novae**  |
|| A. Chong, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-01-09*|
|*Comments*| *23 pages, 58 figures, 1 table, submitted to MNRAS*|
|**Abstract**|            While nova eruptions produce some of the most common and dramatic dust formation episodes among astrophysical transients, the demographics of dust-forming novae remain poorly understood. Here, we present a statistical study of dust formation in 40 novae with high-quality optical/IR light curves, quantitatively distinguishing dust-forming from non-dust-forming novae while exploring the properties of the dust events. We find that 50-70% of novae produce dust, significantly higher than previous estimates. Dust-forming novae can be separated from those that do not show dust formation by using the largest redward ($V-K$) colour change from peak visible brightness; ($V-J$) or ($V-H$) offer useful but less sensitive constraints. This makes optical+IR photometry a powerful tool to quantify dust formation in novae. We find that novae detected in GeV $\gamma$-rays by \emph{Fermi}-LAT appear to form dust more often than novae not detected by \emph{Fermi}, implying a possible connection between $\gamma$-ray producing shocks and dust production. We also find that novae that evolve very quickly ($t_2 < 10$ days) are much less likely to form dust, in agreement with previous findings. We confirm a correlation between $t_2$ and the time of the onset of dust formation (which occurs $\sim$1 week--3 months after maximum light), but conclude that it is primarily an observational artifact driven by dust formation determining when a nova drops 2 mag below peak. The significant fraction of novae that form dust make them ideal laboratories in our Galactic backyard to tackle the puzzle of dust formation around explosive transients.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.04518/./figures/moment_maps/HCOp1-0_mom0_merged.png', 'tmp_2501.04518/./figures/moment_maps/N2Hp1-0_mom0.png', 'tmp_2501.04518/./figures/moment_maps/SiO2-1_mom0.png', 'tmp_2501.04518/./figures/moment_maps/NH2D_mom0.png', 'tmp_2501.04518/./figures/rgb_images/DR18_rgb_hcop_8micron_6cm_new.png', 'tmp_2501.04518/./figures/images/DR18_brgamma_h2_diff_fullmap_zoom_sio_n2hp.png', 'tmp_2501.04518/./figures/images/vla_noema_continuum.png']
copying  tmp_2501.04518/./figures/moment_maps/HCOp1-0_mom0_merged.png to _build/html/
copying  tmp_2501.04518/./figures/moment_maps/N2Hp1-0_mom0.png to _build/html/
copying  tmp_2501.04518/./figures/moment_maps/SiO2-1_mom0.png to _build/html/
copying  tmp_2501.04518/./figures/moment_maps/NH2D_mom0.png to _build/html/
copying  tmp_2501.04518/./figures/rgb_images/DR18_rgb_hcop_8micron_6cm_new.png to _build/html/
copying  tmp_2501.04518/./figures/images/DR18_brgamma_h2_diff_fullmap_zoom_sio_n2hp.png to _build/html/
copying  tmp_2501.0451

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\mbox{km s^{-1}}}$
$\newcommand{\hi}{H{\sc i}}$
$\newcommand{\hii}{H{\sc ii}}$</div>



<div id="title">

# The Cygnus Allscale Survey of Chemistry and Dynamical Environments: CASCADE: IV. Unveiling the hidden structures in DR18

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.04518-b31b1b.svg)](https://arxiv.org/abs/2501.04518)<mark>Appeared on: 2025-01-09</mark> -  _25 pages, 21 figures, and 4 tables. Accepted for publication in A&A_

</div>
<div id="authors">

W.-J. Kim, et al. -- incl., <mark>H. Beuther</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** The Cygnus-X complex is a massive (a few $10^6$ M $_{\odot}$ molecular gas mass), nearby (1.4 kpc) star-forming region with several OB associations. Of these, Cyg OB2 is the largest,  with at least 169 OB stars. DR18 is the largest globule near the OB2 association, making it a perfect target for investigating the influence of ultraviolet radiation on molecular clouds. By analyzing emission from different molecular species, we aim to study the molecular gas structures toward DR18 using high angular-resolution molecular line observations. As part of the Cygnus Allscale Survey of Chemistry and Dynamical Environments (CASCADE) program,  we carried out 3.6 millimeter (mm) continuum and spectral line high-resolution observations ( $\sim$ 3 -- 4 $"$ ) toward DR18, covering several molecular species (e.g., HCN, HNC, $\ce{H2CO}$ , $\ce{N2H+}$ , SiO, $\ce{C2H}$ , deuterated species, etc.) with the Northern Extended Millimeter Array (NOEMA) and the Institut de Radioastronomie Millimétrique (IRAM) 30 m telescope. In addition, multi-wavelength archival datasets from mid-infrared (MIR) to centimeter (cm) wavelengths were used to provide a comprehensive analysis of the region. The spectral index analysis shows significant contamination of the 3.6  mm continuum by free-free emission from ionized gas. A comparison of the 3.6 mm and 6 cm continuum emission confirms that a B2 star (DR18-05) shapes the cometary $\hii$ region in the DR18 cavity, with ionized gas escaping toward the OB2 association. On the other hand, the extended 3.6 mm and 6 cm continuum emission are likely to trace photoevaporating ionized gas from ultraviolet radiation from the Cyg OB2 association --  not from DR18-05. To study the feedback of the B2 star and the OB2 association on surrounding molecular regions, we analyzed the $\ce{HCO+}$ , HCN, HNC, $\ce{N2H+}$ , and SiO emission lines. The shell structure around DR18-05 indicates photodissociation regions (PDRs) formed by the expanding $\hii$ region and photo-erosion from DR18-05 and OB2 stars. We also identified 18 compact cores with $\ce{N2H+}$ emission, half of which are gravitationally bound (virial parameter, $\alpha_{\rm vir}$ , $\lesssim$ 2.0), and mostly located in colder regions ( $T_{\rm HCN/HNC}$ $<$ 30 K) behind the PDRs. The SiO emission is found only in PDRs, with narrow-line widths ( $\sim$ 0.8 -- 2.0 $\kms$ ) and lower abundances (X(SiO) $\sim 5\times10^{-11} - 1 \times10^{-10}$ ). Comparing with the UV irradiated shock models, we suggest that the SiO emission partially encompassing the $\hii$ region arises from the molecular gas region, marginally compressed by low-velocity shocks with $\sim$ 5 $\kms$ , irradiated by external UV radiation ( $G_{\rm 0} \sim 10^2 - 10^3$ ), as they traverse through a medium with $n_{\rm H} \sim 10^4$ to $10^5$ cm $^{-3}$ . These shocks can be generated by the initial expansion of the $\hii$ region and potentially by stellar winds.

</div>

<div id="div_fig1">

<img src="tmp_2501.04518/./figures/moment_maps/HCOp1-0_mom0_merged.png" alt="Fig15.1" width="25%"/><img src="tmp_2501.04518/./figures/moment_maps/N2Hp1-0_mom0.png" alt="Fig15.2" width="25%"/><img src="tmp_2501.04518/./figures/moment_maps/SiO2-1_mom0.png" alt="Fig15.3" width="25%"/><img src="tmp_2501.04518/./figures/moment_maps/NH2D_mom0.png" alt="Fig15.4" width="25%"/>

**Figure 15. -** Velocity-integrated intensity maps of \ce{HCO+}(6 $\kms$ to 15.6 $\kms$), \ce{N2H+}($-$1.2 $\kms$ to 17.2 $\kms$ spanning all the hfs transitions) in the upper row and SiO (8 $\kms$ to 10 $\kms$) and \ce{NH2D}(0 $\kms$ to 15.2 $\kms$ covering all the hfs transitions) in the lower row. The black contours represent the GBT+NOEMA combined 3.6 mm continuum emission, and the bright green marker indicates the position of the ionizing source, DR18-05. The contour levels of the mm continuum are the same as shown in Fig. \ref{fig:mm_cm_images}. The horizontal white scale bar has a size of 0.1 pc at a given distance of 1.4 kpc. The beam of the maps of each species is displayed on the left corner of each image panel.  (*fig:mom0_maps*)

</div>
<div id="div_fig2">

<img src="tmp_2501.04518/./figures/rgb_images/DR18_rgb_hcop_8micron_6cm_new.png" alt="Fig17.1" width="50%"/><img src="tmp_2501.04518/./figures/images/DR18_brgamma_h2_diff_fullmap_zoom_sio_n2hp.png" alt="Fig17.2" width="50%"/>

**Figure 17. -** Different emission distributions toward DR18. _Left_: Three-color composite image of DR18; \ce{HCO+} emission in red, 8 $\mu$m emission in green, and the GLOSTAR 6 cm radio continuum emission in blue. The 6 cm emission represents a combination VLA D- and B-configuration data from the GLOSTAR survey. The synthesized beam size of the GLOSTAR data is $4"$,  comparable to that of NOEMA data ($\sim 3-4"$). The pixel size,  of the _Spitzer_/IRAC 8 $\mu$m data is $0\rlap{.}"6$. The black dashed lines indicate the cuts for intensity profiles of different emission tracers, presented in Fig. \ref{fig:intensity_profile}. The white rectangles represent the areas of the East and North shells, as well as the Ridge. _Right_: Flux difference image of Br$\gamma$ and \ce{H2} 1--0 $S$(1) emission lines obtained from [Comerón, Schneider and Djupvik (2022)](). The black and yellow contours show the 5$\sigma$ level of \ce{N2H+} velocity-integrated intensity and a 3.0$\sigma$ level of SiO velocity-integrated intensity, respectively. In both images, the bright green star indicates DR18-05, the ionizing source in DR18.  (*fig:dr18_pdr_image*)

</div>
<div id="div_fig3">

<img src="tmp_2501.04518/./figures/images/vla_noema_continuum.png" alt="Fig13" width="100%"/>

**Figure 13. -** Millimeter (left) and centimeter (right) continuum emission maps. The mm continuum emission map represents the combination of the GBT MUSTANG2 and NOEMA data sets, whereas the cm continuum emission map is generated by combining the VLA D and B configuration observations. The FWHM synthetic beams of these maps are displayed in the lower left corners. The contour levels of the mm continuum emission are 0.46, 0.96, 1.38, 1.80, 4.14, 5.33, and 6.51 mJy beam$^{-1}$, whereas for the cm emission, the levels are 0.35, 0.70, 1.05, 1.40, 1.75, 2.10, 2.45, 2.80, 3.15, and 3.50 mJy beam$^{-1}$. The star symbol indicates the position of the B2 star. (*fig:mm_cm_images*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.04518"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kepler}{{\it Kepler}}$
$\newcommand{\corot}{{\it CoRoT}}$
$\newcommand{\TESS}{{\it TESS}}$
$\newcommand{\tess}{{\it TESS}}$
$\newcommand{\PLATO}{{\it PLATO}}$
$\newcommand{\plato}{{\it PLATO}}$
$\newcommand{\gaia}{{\it Gaia}}$
$\newcommand{\GAIA}{{\it Gaia}}$
$\newcommand{\twomass}{{\it 2MASS}}$
$\newcommand{\JWST}{{\it JWST}}$
$\newcommand{\jwst}{{\it JWST}}$
$\newcommand{\NGTS}{{\it NGTS}}$
$\newcommand{\ngts}{{\it NGTS}}$
$\newcommand{\LSO}{La Silla Observatory}$
$\newcommand{\paranal}{{Paranal Observatory}}$
$\newcommand{\Spitzer}{\textit{Spitzer}}$
$\newcommand{\HARPS}{{\it HARPS}}$
$\newcommand{\harps}{{\it HARPS}}$
$\newcommand{\Minerva}{{\it MINERVA-Australis}}$
$\newcommand{\minerva}{{\it MINERVA-Australis}}$
$\newcommand{\iSHELL}{{\it iSHELL}}$
$\newcommand{\ishell}{{\it iSHELL}}$
$\newcommand{\FEROS}{{\it FEROS}}$
$\newcommand{\feros}{{\it FEROS}}$
$\newcommand{\CORALIE}{{\it CORALIE}}$
$\newcommand{\coralie}{{\it CORALIE}}$
$\newcommand{\Ktwo}{K2}$
$\newcommand{\hires}{{\it HIRES}}$
$\newcommand{\HiRES}{{\it HIRES}}$
$\newcommand{\HIRES}{{\it HIRES}}$
$\newcommand{\pfs}{{\it PFS}}$
$\newcommand{\PFS}{{\it PFS}}$
$\newcommand{\MEarth}{{\it MEarth}}$
$\newcommand{\MEarthSouth}{{\it MEarth-South}}$
$\newcommand{\astep}{{\it ASTEP}}$
$\newcommand{\ASTEP}{{\it ASTEP}}$
$\newcommand{\lco}{{\it LCOGT}}$
$\newcommand{\lcogt}{{\it LCOGT}}$
$\newcommand{\LCO}{{\it LCOGT}}$
$\newcommand{\LCOGT}{{\it LCOGT}}$
$\newcommand{\waspsouth}{{\it WASP-South}}$
$\newcommand{\wasp}{{\it WASP}}$
$\newcommand{\WASP}{{\it WASP}}$
$\newcommand{\kelt}{{\it KELT}}$
$\newcommand{\hatnet}{{\it HATNet}}$
$\newcommand{\hatsouth}{{\it HAT-South}}$
$\newcommand{\cheops}{{\it CHEOPS}}$
$\newcommand{\CHEOPS}{{\it CHEOPS}}$
$\newcommand{\Keck}{{\it Keck}}$
$\newcommand{\keck}{{\it Keck}}$
$\newcommand{\Gemini}{{\it Gemini}}$
$\newcommand{\gemini}{{\it Gemini}}$
$\newcommand{\VLT}{{\it VLT}}$
$\newcommand{\vlt}{{\it VLT}}$
$\newcommand{\SOAR}{{\it SOAR}}$
$\newcommand{\soar}{{\it SOAR}}$
$\newcommand{\tres}{{\it TRES}}$
$\newcommand{\TRES}{{\it TRES}}$
$\newcommand{\kesprint}{{\it KESPRINT}}$
$\newcommand{\KESPRINT}{{\it KESPRINT}}$
$\newcommand{\ucac}{{\it UCAC4}}$
$\newcommand{\apass}{{\it APASS}}$
$\newcommand{\saao}{{\it SAAO}}$
$\newcommand{\SAAO}{{\it SAAO}}$
$\newcommand{\eulercam}{{\it EulerCam}}$
$\newcommand{\trappist}{{\it TRAPPIST}}$
$\newcommand{\TRAPPIST}{{\it TRAPPIST}}$
$\newcommand{\elsauce}{{\it El Sauce}}$
$\newcommand{\speculoos}{{\it SPECULOOS}}$
$\newcommand{\nirps}{{\it NIRPS}}$
$\newcommand{\espresso}{{\it ESPRESSO}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\msy}{m s^{-1} yr^{-1}}$
$\newcommand{\masy}{mas yr^{-1}}$
$\newcommand{\wm}{W m^{-2}}$
$\newcommand{\tc}{T\textsubscript{\textit{c}}}$
$\newcommand{\texp}{\mbox{t_{exp}}}$
$\newcommand{\mpl}{\mbox{M_{p}}}$
$\newcommand{\rpl}{\mbox{R\textsubscript{p}}}$
$\newcommand{\mstar}{\mbox{M_{\star}}}$
$\newcommand{\rstar}{\mbox{R_{\star}}}$
$\newcommand{\lstar}{\mbox{L_{\star}}}$
$\newcommand{\mjup}{\mbox{M\textsubscript{J}}}$
$\newcommand{\rjup}{\mbox{R\textsubscript{J}}}$
$\newcommand{\msun}{\mbox{M_{\odot}}}$
$\newcommand{\rsun}{\mbox{R_{\odot}}}$
$\newcommand{\lsun}{\mbox{L_{\odot}}}$
$\newcommand{\ssun}{\mbox{S_{\odot}}}$
$\newcommand{\Lsun}{\mbox{L_{\odot}}}$
$\newcommand{\Msun}{\mbox{M_{\odot}}}$
$\newcommand{\Rsun}{\mbox{R_{\odot}}}$
$\newcommand{\Ssun}{\mbox{S_{\odot}}}$
$\newcommand{\rearth}{R_{\oplus}}$
$\newcommand{\mearth}{M_{\oplus}}$
$\newcommand{\gccc}{g cm^{-3}}$
$\newcommand{\ergscm}{erg s^{-1}cm^{-2}}$
$\newcommand{\teff}{T\textsubscript{eff}}$
$\newcommand{\tmag}{T\textsubscript{mag}}$
$\newcommand{\teq}{T_{\rm eq}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\microm}{\mum}$
$\newcommand{\mone}{\mbox{M_\text{A}}}$
$\newcommand{\rone}{\mbox{R_\text{A}}}$
$\newcommand{\mtwo}{\mbox{M_\text{B}}}$
$\newcommand{\rtwo}{\mbox{R_\text{B}}}$
$\newcommand{\vsini}{v \sin i_\star}$
$\newcommand{\vrad}{v_{\rm rad}}$
$\newcommand{\vmic}{v_{\rm mic}}$
$\newcommand{\vmac}{v_{\rm mac}}$
$\newcommand{\dex}{\rm dex}$
$\newcommand{\met}{[M/H]}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\mgh}{[Mg/H]}$
$\newcommand{\sih}{[Si/H]}$
$\newcommand{\ch}{[C/H]}$
$\newcommand{\oh}{[O/H]}$
$\newcommand{\mdot}{\.{M}}$
$\newcommand{\msunyr}{M_{\odot} yr^{-1}}$
$\newcommand{\MJ}{M_{J}}$
$\newcommand{\RJ}{R_{J}}$
$\newcommand{\halpha}{H\alpha}$
$\newcommand{\hbeta}{H\beta}$
$\newcommand{\sii}{[S {\sc II}] }$
$\newcommand{\oii}{[O {\sc I}] }$
$\newcommand{\oiii}{[O {\sc III}] }$
$\newcommand{\oi}{[O {\sc i}] }$
$\newcommand{\cai}{Ca {\sc I} }$
$\newcommand{\caii}{[Ca {\sc II}] }$
$\newcommand{\feii}{[Fe {\sc II}] }$
$\newcommand{\fei}{[Fe {\sc I}] }$
$\newcommand{\nii}{[N {\sc II}] }$
$\newcommand{\mgi}{Mg {\sc I} }$
$\newcommand{\nai}{[Na {\sc I}]}$
$\newcommand{\msd}{m~s^{-1}~d^{-1}}$
$\newcommand{\gc}{g~cm^{-3}}$
$\newcommand{\lgr}{\log (R^\prime_{HK})}$
$\newcommand{\lgt}{\log~t}$
$\newcommand{\logrhk}{log R^\prime_\mathrm{HK}}$
$\newcommand{\tiara}{\texttt{TIaRA}}$
$\newcommand{\TICID}{TIC 238060327}$
$\newcommand{\OBJID}{\ngts-EB-7}$
$\newcommand{\logrhkval}{-5.12\pm0.17}$
$\newcommand{\secondarydurtot}{12.7\pm0.2}$
$\newcommand{\secondarydurfull}{10.6\pm0.2}$
$\newcommand{\Tnoughtocc}{2459503.5\pm0.1}$
$\newcommand{\phinoughtocc}{0.0900\pm0.0006}$
$\newcommand{\MBMsun}{0.096^{+0.003}_{-0.004}}$
$\newcommand{\MBMjup}{102^{+4}_{-5}}$
$\newcommand{\apodistance}{1.13\pm0.06}$
$\newcommand{\peridistance}{0.189\pm0.009}$
$\newcommand{\hostteff}{5770\pm110}$
$\newcommand{\hostlogg}{4.17\pm0.15}$
$\newcommand{\hostmet}{0.26\pm0.12}$
$\newcommand{\hostvsini}{2.7\pm0.8}$
$\newcommand{\hostrad}{1.45^{+0.07}_{-0.06}}$
$\newcommand{\isohostmass}{1.13^{+0.06}_{-0.07}}$
$\newcommand{\hostage}{10\pm1}$
$\newcommand{\alles}{\texttt{allesfitter}}$
$\newcommand{\paws}{\texttt{PAWS}}$
$\newcommand{\ariadne}{\texttt{astroARIADNE}}$
$\newcommand{\specmatch}{\texttt{specmatch-emp}}$
$\newcommand{\isochrones}{\texttt{isochrones}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# $\OBJID$, an eccentric, long-period, low-mass eclipsing binary

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.04523-b31b1b.svg)](https://arxiv.org/abs/2501.04523)<mark>Appeared on: 2025-01-09</mark> -  _Main body: 14 pages, 6 figures, 5 tables. Appendices: 7 pages, 5 figures, 5 tables_

</div>
<div id="authors">

T. Rodel, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Despite being the most common types of stars in the Galaxy, the physical properties of late M dwarfs are often poorly constrained. A trend of radius inflation compared to evolutionary models has been observed for earlier type M dwarfs in eclipsing binaries, possibly caused by magnetic activity. It is currently unclear whether this trend also extends to later type M dwarfs below the convective boundary. This makes the discovery of lower-mass, fully convective, M dwarfs in eclipsing binaries valuable for testing evolutionary models – especially in longer-period binaries where tidal interaction between the primary and secondary is negligible. With this context, we present the discovery of the $\OBJID$ AB system, an eclipsing binary containing a late M dwarf secondary and an evolved G-type primary star. The secondary star has a radius of $\BRcompanionRsun$ $\rsun$ , a mass of $\MBMsun$ $\msun$ and follows a highly eccentric (e= $\Be$ ) orbit every $\Bperiod$ days. This makes $\OBJID$ AB the third longest-period eclipsing binary system with a secondary smaller than 200 $\mjup$ with the mass and radius constrained to better than 5 \% . In addition, $\OBJID$ is situated near the centre of the proposed LOPS2 southern field of the upcoming $\plato$ mission, allowing for detection of the secondary eclipse and measurement of the companion's temperature. With its long-period and well-constrained physical properties - $\OBJID$ B will make a valuable addition to the sample of M dwarfs in eclipsing binaries and help in determining accurate empirical mass/radius relations for later M dwarf stars.

</div>

<div id="div_fig1">

<img src="tmp_2501.04523/./Figures/RV_plot_time.png" alt="Fig2.1" width="50%"/><img src="tmp_2501.04523/./Figures/RV_plot_phase.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Radial velocity measurements of $\OBJID$.Radial velocity measurements of $\OBJID$. Data points from $\coralie$ are denoted with a black circle, $\harps$ is shown with a blue square and $\feros$ with grey triangles. The median fitted RV model is also overplotted in red. The top panel shows the data while the bottom shows the residuals after subtracting the best model fit. Subfigure **(a)** shows the data plotted versus time while **(b)** shows the same data as a function of orbital phase.Radial velocities as a function of time.Radial velocities after phase-folding on the orbital period of \Bperiod days. (*fig:RV_plot*)

</div>
<div id="div_fig2">

<img src="tmp_2501.04523/./Figures/lightcurves/TESS8_transit.png" alt="Fig1.1" width="33%"/><img src="tmp_2501.04523/./Figures/lightcurves/TESS29_transit.png" alt="Fig1.2" width="33%"/><img src="tmp_2501.04523/./Figures/lightcurves/NGTS_transit.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Transit lightcurves of $\OBJID$ normalised to the out-of-transit flux levels. Each panel shows the median transit model (see Section \ref{section:allesfitter}) as a solid red line. The transit data plotted is shown as black square markers with errorbars. The top panel of each subfigure shows the data and model while the lower panel of each shows the residuals after the model has been subtracted from the observed data.$\tess$ Sector 8 transit.$\tess$ Sector 29 transit.$\ngts$ transit. (*fig:transits*)

</div>
<div id="div_fig3">

<img src="tmp_2501.04523/./Figures/gaia_HRD.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\gaia$ DR3 Hertzsprung Russell diagram with the position of $\OBJID$ A highlighted with a black star symbol. $\gaia$ BP-RP colour is plotted against absolute magnitude in the G band for all stars in the crossmatched SPOC FFI sample described in \protect[Doyle, et. al (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.529.1802D). The position of $\OBJID$ A shown is consistent with a star beginning to move off the main sequence. (*fig:gaia_HRD*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.04523"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# MINDS. The influence of outer dust disc structure on the volatile delivery to the inner disc

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.04587-b31b1b.svg)](https://arxiv.org/abs/2501.04587)<mark>Appeared on: 2025-01-09</mark> -  _Accepted for publication in A&A. 29 pages, 12 figures and 10 tables_

</div>
<div id="authors">

D. Gasman, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The Atacama Large Millimeter/submillimeter Array (ALMA) has revealed that the millimetre dust structures of protoplanetary discs are extremely diverse, ranging from small and compact dust discs to large discs with multiple rings and gaps. It has been proposed that the strength of $\ce{H2O}$ emission in the inner disc particularly depends on the influx of icy pebbles from the outer disc, a process that would correlate with the outer dust disc radius, and that could be prevented by pressure bumps. Additionally, the dust disc structure should also influence the emission of other gas species in the inner disc. Since terrestrial planets likely form in the inner disc regions, understanding their composition is of interest. This work aims to assess the influence of pressure bumps on the inner disc's molecular reservoirs. The presence of a dust gap, and potentially giant planet formation farther out in the disc, may influence the composition of the inner disc, and thus the building blocks of terrestrial planets. Using the improved sensitivity and spectral resolution of the Mid-InfraRed Instrument's (MIRI) Medium Resolution Spectrometer (MRS) on the _James Webb_ Space Telescope ( _JWST_ ) compared to _Spitzer_ , we compared the observational emission properties of $\ce{H2O}$ , $\ce{HCN}$ , $\ce{C2H2}$ , and $\ce{CO2}$ with the outer dust disc structure from ALMA observations, in eight discs with confirmed gaps in ALMA observations, and two discs with gaps of tens of astronomical units in width, around stars with $M_\star \geq 0.45M_{\odot}$ . We used new visibility plane fits of the ALMA data to determine the outer dust disc radius and identify substructures in the discs. We find that the presence of a dust gap does not necessarily result in weak $\ce{H2O}$ emission. Furthermore, the relative lack of colder $\ce{H2O}$ -emission seems to go hand in hand with elevated emission from carbon-bearing species. Of the discs that show significant substructure within the $\ce{CO}$ and $\ce{CH4}$ snowlines, most show detectable emission from the carbon-bearing species. The discs with cavities and extremely wide gaps appear to behave as a somewhat separate group, with stronger cold $\ce{H2O}$ emission and weak warm $\ce{H2O}$ emission. We conclude that fully blocking radial dust drift from the outer disc seems difficult to achieve, even for discs with very wide gaps or cavities, which can still show significant cold $\ce{H2O}$ emission. However, there does seem to be a dichotomy between discs that show a strong cold $\ce{H2O}$ excess and ones that show strong emission from $\ce{HCN}$ and $\ce{C2H2}$ . Better constraints on the influence of the outer dust disc structure and inner disc composition require more information on substructure formation timescales and disc ages, along with the importance of trapping of (hyper)volatiles like CO and $\ce{CO2}$ into more strongly bound ices like $\ce{H2O}$ and chemical transformation of CO into less volatile species.

</div>

<div id="div_fig1">

<img src="tmp_2501.04587/./figures/scenario_schematic.png" alt="Fig8" width="100%"/>

**Figure 8. -** Emission properties of the inner disc based on the modelling works by \citet{ref:21KaPiKr}, \citet{ref:23KaPiKr}, \citet{ref:24MaSaBi}, \citet{ref:24LiBiHe}, and \citet{ref:24SeVlDi}. (*fig:scenarios*)

</div>
<div id="div_fig2">

<img src="tmp_2501.04587/./figures/radial_profiles.png" alt="Fig6" width="100%"/>

**Figure 6. -** Radial profiles of the disc sample based on the analysis in the image (black line) and visibility plane (grey line). The snowline estimates of \ce{H2O}, \ce{CO2}, \ce{CH4}, and \ce{CO} are indicated by the coloured areas. The minimum distance from the star that can be imaged, based on half the beam radius, is indicated by the hatched area. The $R_\text{gap}$, or the location of the innermost detected local minimum, is indicated by the vertical dotted line. The discs are organised from strongest \ce{H2O}/$L_\text{acc}^{0.6}$(top left) to weakest (bottom right), with the two (pre-)transition discs in the bottom row. (*fig:radial_profiles*)

</div>
<div id="div_fig3">

<img src="tmp_2501.04587/./figures/hot_cold.png" alt="Fig1" width="100%"/>

**Figure 1. -** Average of integrated line fluxes of warm (top, orange) and cold (middle, blue) \ce{H2O} gas, and their ratio (bottom, black), plotted against the outer dust disc radius. The PCCs and p values for these log-log scatter plots are given in the panels, and the corresponding trend is indicated by the dotted black line. Upper and lower limits are indicated by triangles. (*fig:water_radius_corr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.04587"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gaia}{\textsl{Gaia }}$
$\newcommand{\gaianospace}{\textit{Gaia}}$
$\newcommand{\kms}{km~s^{-1} }$
$\newcommand{\kmsnospace}{km~s^{-1}}$
$\newcommand{◦}{^{\circ}~}$
$\newcommand{◦nospace}{^{\circ}}$
$\newcommand{\addreferences}{\textcolor{red}{(add References) }}$
$\newcommand{\msun}{M_\odot }$
$\newcommand{\vlos}{V_{\text{los}} }$
$\newcommand{\vlosnospace}{V_{\text{los}}}$
$\newcommand{\sourceid}{\texttt{source\_id} }$
$\newcommand{\sourceidnospace}{\texttt{source\_id}}$
$\newcommand{\danny}[1]{{\textcolor{blue}{ #1}}}$
$\newcommand{\oscar}[1]{{\textcolor{brown}{ #1}}}$
$\newcommand{\action}[1]{{\textcolor{magenta}{ [Action for: #1]}}}$
$\newcommand{\kmskpc}{km~s^{-1}~kpc^{-1} }$
$\newcommand{\kmskpcnospace}{km~s^{-1}~kpc^{-1}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Vertical structure and kinematics of the LMC disc from \textsl{SDSS}/\textsl{Gaia}

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.04616-b31b1b.svg)](https://arxiv.org/abs/2501.04616)<mark>Appeared on: 2025-01-09</mark> -  _Submitted to A&A. The interactive LMC 3D models can be found in this https URL_

</div>
<div id="authors">

Ó. Jiménez-Arranz, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** Studies of the LMC's internal kinematics have provided a detailed view of its structure, largely thanks to the exquisite proper motion data supplied by the $\gaia$ mission. However, line-of-sight (LoS) velocities, the third component of the stellar motion, are only available for a small subset of the current $\gaia$ data, limiting studies of the kinematics perpendicular to the LMC disc plane. We synergise new \textsl{SDSS} -IV/V LoS velocity measurements with existing $\gaia$ DR3 data, increasing the 5D phase-space sample by almost a factor of three. Using this unprecedented dataset, we interpret and model the vertical structure and kinematics across the LMC disc. We first split our parent sample into different stellar types (young and old). We then examine maps of vertical velocity, $v_{z'}$ , moments (median and MAD) perpendicular to the LMC disc out to $R' \approx 5$ kpc; we also examine the vertical velocity profiles as a function of disc azimuth and radius. We interpret our results in the context of three possible scenarios: 1) time-variability in the orientation of the disc symmetry axis; 2) use of an incorrect LMC disc plane orientation; or 3) the presence of warps or twists in the LMC disc. We also present a new inversion method to construct a continuous 3D representation of the disc from spatially-resolved measurements of its viewing angles. Using young stellar populations, we identify a region in the LMC arm with highly negative $\overline{v_{z'}}$ ; this overlaps spatially with the supershell LMC 4. When interpreting the maps of $\overline{v_{z'}}$ , our results indicate that: 1) the LMC viewing angles may vary with time due to, e.g., precession or nutation of the spin axis, but this cannot explain most of the structure in $\overline{v_{z'}}$ maps;2) when re-deriving the LMC disc plane by minimising the RMS vertical velocity $v_{z'}$ across the disc, the inclination and line-of-nodes position angle are $i = 24^\circ$ and $\Omega = 327^\circ$ , respectively, with an $\sim 3^\circ$ systematic uncertainty associated with sample selection, contamination, and the position of the LMC center; 3) when modelling in concentric rings, we obtain different inclinations for the inner and outer disc regions, and when modelling in polar segments we obtain a quadrupolar variation as function of azimuth in outer the disc. We provide 3D representations of the implied LMC disc shape. These provide further evidence for  perturbations caused by interaction with the SMC. The combination of \textsl{SDSS} -IV/V and $\gaia$ data reveal that the LMC disc is not a flat plane in equilibrium, but that the central bar region is tilted relative to a warped outer disc.

</div>

<div id="div_fig1">

<img src="tmp_2501.04616/./figures/01_LMC_vertical_velocity_maps_eDR3_muzfree.png" alt="Fig8" width="100%"/>

**Figure 8. -** Comparison of the median vertical velocity maps between the different LMC clean samples. Top: LMC complete sample. Bottom: LMC optimal sample. From left to right: \textsl{APOGEE}, \textsl{BOSS}, $\gaia$nospace, and Combined sample. We display bins containing three or more stars; otherwise, we display individual stars as a scatter plot. A black line splitting the overdensities (LMC bar and spiral arm) from the underdensities is plotted for reference. All maps are shown in the LMC in-plane $(x', y')$ Cartesian coordinate system. To mimic how the LMC is seen in the sky, the plotted data has both axes inverted.
     (*fig:samples_vertical_velocity*)

</div>
<div id="div_fig2">

<img src="tmp_2501.04616/./figures/08_Radial_variation_LMC_disc_fitting.png" alt="Fig5" width="100%"/>

**Figure 5. -** Variation of the LMC disc plane fitting as function of the LMC galactocentric radius $R'$. Top: Inclination $i$. Bottom: Line-of-nodes position angle $\Omega$. The "global" fit derived in Sect. \ref{subsec:incorrect_plane} is displayed by the horizontal dashed lines. We use a grey shaded area to draw attention to the bar region \citep[$R'_{\text{bar}}=2.3$ kpc,][]{jimenez-arranz23a}. In all panels we display the LMC complete (optimal) sample in green (salmon). (*fig:disc_fit_vs_radius*)

</div>
<div id="div_fig3">

<img src="tmp_2501.04616/./figures/09_Azimuthal_radial_variation_LMC_disc_fitting.png" alt="Fig12" width="100%"/>

**Figure 12. -** Variation of the LMC disc plane fitting as function of the azimuth $\phi$ and the LMC galactocentric radius $R'$. Left panels: LMC Combined complete sample. Right panels: LMC Combined optimal sample. The first and the third (second and fourth) panels show the inclination $i$(line-of-nodes position angle $\Omega$). Only bins with more than 50 stars are displayed. Each colorbar is centred in the "global" fit derived in Sect. \ref{subsec:incorrect_plane}. (*fig:disc_fit_vs_azimuth*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.04616"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

299  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
